# 2023 COMP 4446 / 5046 Assignment 1

Assingment 1 is an **individual** assessment. Please note the University's [Academic dishonesty and plagiarism policy](https://www.sydney.edu.au/students/academic-dishonesty.html).

Submission Deadline: Friday, March 17th, 2023, 11:59pm

Submit via Canvas:
- Your notebook
- Run all cells before saving the notebook, so we can see your output

In this assignment, we will explore ways to predict the length of a Wikipedia article based on the first 100 tokens in the article. Such a model could be used to explore whether there are systematic biases in the types of articles that get more detail.

If you are working in another language, please make sure to clearly indicate which part of your code is running which section of the assignment and produce output that provides all necessary information. Submit your code, example outputs, and instructions for executing it.

Note: This assignment contains topics that are not covered at the time of release. Each section has information about which lectures and/or labs covered the relevant material. We are releasing it now so you can (1) start working on some parts early, and (2) know what will be in the assignment when you attend the relevant labs and lectures.

# **TODO: Copy and Name this File**
Make a copy of this notebook in your own Google Drive (File -> Save a Copy in Drive) and change the filename, replacing `YOUR-UNIKEY`. For example, for a person with unikey `mcol1997`, the filename should be:

`COMP-4446-5046_Assignment1_mcol1997.ipynb`

# Readme
*If there is something you want to tell the marker about your submission, please mention it here.* 



```
# This is formatted as code
```
In this assignment for the pre-processing of the training text for section 1-2 i just split it based on the space. In total i generated 7739 tokens from the training text after splitting it based on the filter of occurring atleast 10 times. In this 10 is included in the calculation. 
For section 3 the entire training text was used after basic regex pre-processing.


# Data Download [DO NOT MODIFY THIS]

We have already constructed a dataset for you using a recent dump of data from Wikipedia. Both the training and test datasets are provided in the form of csv files (training_data.csv, test_data.csv) and can be downloaded from Google Drive using the code below. Each row of the data contains:

- The length of the article
- The title of the article
- The first 100 tokens of the article

In case you are curious, we constructed this dataset as follows:
1. Downloaded [a recent dump](https://dumps.wikimedia.org/) of English wikipedia.
2. Ran [WikiExtractor](https://github.com/attardi/wikiextractor) to get the contents of the pages.
3. Filtered out very short pages.
4. Ran [SpaCy](https://spacy.io/) with the `en_core_web_lg` model to tokenise the pages (Note, SpaCy's development is led by an alumnus of USyd!).
5. Counted the tokens and saved the relevant data in the format described above.

This code will download the data. **DO NOT MODIFY IT**

In [1]:
## DO NOT MODIFY THIS CODE
# Code to download files into Colaboratory

# Install the PyDrive library
!pip install -U -q PyDrive

# Import libraries for accessing Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Function to read the file, save it on the machine this colab is running on, and then read it in
import csv
def read_file(file_id, filename):
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.GetContentFile(filename)
  with open(filename) as src:
    reader = csv.reader(src)
    data = [r for r in reader]
  return data

# Calls to get the data
# If you need to access the data directly (e.g., you are running experiments on a local machine), use these links:
# - Training, https://drive.google.com/file/d/1-UGFS8D-qglAX-czU38KaM4jQVCoNe0W/view?usp=share_link
# - Dev, https://drive.google.com/file/d/1RWMEf0mdJMTkWc7dvN0ioks8bjujqZaN/view?usp=share_link
# - Test, https://drive.google.com/file/d/1YVPNzdIFSMmVPeLBP-gf5DOIed3oRFyB/view?usp=share_link
training_data = read_file('1-UGFS8D-qglAX-czU38KaM4jQVCoNe0W', "/content/training_data.csv")
dev_data = read_file('1RWMEf0mdJMTkWc7dvN0ioks8bjujqZaN', "/content/dev_data.csv")
test_data = read_file('1YVPNzdIFSMmVPeLBP-gf5DOIed3oRFyB', "/content/test_data.csv")

print("------------------------------------")
print("Size of training data: {0}".format(len(training_data)))
print("Size of development data: {0}".format(len(dev_data)))
print("Size of test data: {0}".format(len(test_data)))
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1:]))
print("------------------------------------")

# Preview of the data in the csv file, which has three columns: 
# (1) length of article, (2) title of the article, (3) first 100 words in the article
for v in training_data[:10]:
  print("{}\n{}\n{}\n".format(v[0], v[1], v[2][:100] + "..."))

# Store the data in lists and mofidy the length value to be in [0, 1]
training_lengths = [min(1.0, int(r[0])/10000) for r in training_data]
training_text = [r[2] for r in training_data]

dev_lengths = [min(1.0, int(r[0])/10000) for r in dev_data]
dev_text = [r[2] for r in dev_data]

test_lengths = [min(1.0, int(r[0])/10000) for r in test_data]
test_text = [r[2] for r in test_data]


------------------------------------
Size of training data: 9859
Size of development data: 994
Size of test data: 991
------------------------------------
------------------------------------
Sample Data
LABEL: 6453 / SENTENCE: ['Anarchism', 'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy , typically including , though not necessarily limited to , governments , nation states , and capitalism . Anarchism advocates for the replacement of the state with stateless societies or other forms of free associations . As a historically left - wing movement , usually placed on the farthest left of the political spectrum , it is usually described alongside communalism and libertarian Marxism as the libertarian wing ( libertarian socialism )']
------------------------------------
6453
Anarchism
Anarchism is a political philosophy and movement that is ske

# 1 - Predicting article length from initial content

This section relates to content from **the week 1 lecture and the week 2 lab**.

In this section, you will implement training and evaluation of a linear model (as seen in the week 2 lab) to predict the length of a wikipedia article from its first 100 words. You will represent the text using a Bag of Words model (as seen in the week 1 lecture).

## 1.1 Word Mapping [2pt]

In the code block below, write code to go through the training data and for any word that occurs at least 10 times:
- Assign it a unique ID (consecutive, starting at 0)
- Place it in a dictionary that maps from the word to the ID

In [2]:
# Your code goes here
# First splitting the sentences into words. Then we go through the list of words and if it exists in the dictionary then we add one to its countm, otherwise it equals to 1 
#Then we go through the dictionary and store those counts exceeding 10 in another dictionary and then return it.
def word_mapping(training_data):
  new_dict = {}
  for elem in training_data:
    words = elem.split(" ")
    for elem1 in words :
      if elem1 in new_dict :
        new_dict[elem1] += 1
      else : 
        new_dict[elem1] = 1
  filtered_dict = {}
  for key, value in new_dict.items():
    if value >= 10:
      filtered_dict[key] = value
  final_dict = {}
  for k, number in enumerate(filtered_dict):
    final_dict[number] = k
  return final_dict






## 1.2 Data to Bag-of-Words Tensors [2pt]

In the code block below, write code to prepare the data in PyTorch tensors.

The text should be converted to a bag of words (ie., a vector the length of the vocabulary in the mapping in the previous step, with counts of the words in the text).

In [3]:
import torch
from collections import Counter

def bow_converter(t_text, tokens):
  bow_tensor = torch.zeros(len(t_text), len(tokens), dtype=torch.float32)
  for i in range(len(t_text)):
    words = t_text[i].split(" ")
    word_counts = Counter(words)
    for word, count in word_counts.items():
      if word in tokens:
        bow_tensor[i, tokens[word]] = count
  return bow_tensor

x_data = bow_converter(training_text, word_mapping(training_text))
y_data = torch.Tensor(training_lengths)

#We use the same vocabulary from the training_text so that the matrices multiply.
x_dev_data = bow_converter(dev_text, word_mapping(training_text))
y_dev_data = torch.Tensor(dev_lengths)

x_test_data = bow_converter(test_text, word_mapping(training_text))
y_test_data = torch.Tensor(test_lengths)

## 1.3 Model Creation [2pt]

Construct a linear model with an SGD optimiser (we recommend a learning rate of `1e-4`) and mean squared error as the loss.

In [4]:
import torch

import torch.nn as nn
import torch.nn.functional as F

#Mean square error function.
def mse(x1, x2):
  diff = x1 - x2
  return torch.sum(diff*diff)/diff.numel()

linearRegression =  nn.Linear(7739,1)
optimizer = torch.optim.SGD(linearRegression.parameters(), lr=1e-4)

# Define the loss function
loss_func = F.mse_loss

# Calculate loss
loss = loss_func(linearRegression(x_data), y_data.unsqueeze(1))
print(loss)

tensor(0.2853, grad_fn=<MseLossBackward0>)


## 1.4 Training [2pt]

Write a loop to train your model for 100 epochs, printing performance on the dev set every 10 epochs.

In [5]:
no_of_epochs = 100

display_interval = 10

for epoch in range(no_of_epochs):

  predictions = linearRegression(x_data)
  loss = loss_func(predictions, y_data.unsqueeze(1))
  loss.backward()
  training_loss = mse(linearRegression(x_data), y_data.unsqueeze(1))   
  print("Optimised:", "training loss=", "{:.9f}".format(training_loss.data))
  optimizer.step() #call step() to automatically update the parameters through our defined optimizer, which can be called once after backward()
  optimizer.zero_grad() #reset the gradient as what we did before
  if epoch % display_interval == 0 :
      
      # calculate the loss of the current model
      predictions1 = linearRegression(x_dev_data)
      loss = loss_func(predictions1, y_dev_data.unsqueeze(1))          
      print("Epoch:", '%04d' % (epoch), "dev loss=", "{:.8f}".format(loss))


Optimised: training loss= 0.285268039
Epoch: 0000 dev loss= 0.28043595
Optimised: training loss= 0.275471807
Optimised: training loss= 0.266153187
Optimised: training loss= 0.257288843
Optimised: training loss= 0.248856589
Optimised: training loss= 0.240835428
Optimised: training loss= 0.233205289
Optimised: training loss= 0.225947052
Optimised: training loss= 0.219042584
Optimised: training loss= 0.212474629
Optimised: training loss= 0.206226766
Epoch: 0010 dev loss= 0.20529249
Optimised: training loss= 0.200283363
Optimised: training loss= 0.194629580
Optimised: training loss= 0.189251289
Optimised: training loss= 0.184135064
Optimised: training loss= 0.179268092
Optimised: training loss= 0.174638227
Optimised: training loss= 0.170233890
Optimised: training loss= 0.166044101
Optimised: training loss= 0.162058368
Optimised: training loss= 0.158266753
Epoch: 0020 dev loss= 0.15936065
Optimised: training loss= 0.154659748
Optimised: training loss= 0.151228383
Optimised: training loss= 0

## 1.1 Measure Accuracy [2pt]

In the code block below, write code to evaluate your model on the test set.

In [6]:
#First converting the test data into bag of words tensor and the output variable to a tensor.
x_test_data = bow_converter(test_text, word_mapping(training_text))
y_test_data = torch.Tensor(test_lengths)

testing_loss = loss_func(linearRegression(x_test_data), y_test_data.unsqueeze(1)) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))
                


Testing loss= 0.076088734
Absolute mean square loss difference: 0.009256884


The absolute mean square loss difference signifies quite an accurate model.

## 1.2 Analyse the Model [2pt]






In the code block below, write code to identify the 50 words with the highest weights and the 50 words with the lowest weights.

In [7]:
# Your code goes here
numpy_weights = linearRegression.weight.data.detach().numpy()
weights = numpy_weights[0]
#Retrieve the indexes of the top 50 and bottom 50  weights without re-organizing.
top_weights = weights.argsort()[::-1][:50]
lowest_weights = weights.argsort()[:50]
training_words = list(word_mapping(training_text).keys())
print("Top 50 words are :")
for i in top_weights:
    print(training_words[i] + ":" + str(weights[i]))
print("\n")
print("Bottom 50 words are: ")
for i in lowest_weights:
    print(training_words[i] + ":" + str(weights[i]))

Top 50 words are :
,:0.023540823
–:0.011511847
around:0.011445019
border:0.011413005
roughly:0.011399967
rights:0.0113901235
colonial:0.011382417
monarch:0.011377085
figure:0.011366423
Telecom:0.0113570895
complexity:0.011353206
8th:0.01135245
Malaysian:0.011342986
twelfth:0.011336759
behavioral:0.011331191
paramilitary:0.01132567
erotic:0.011325666
Berber:0.0113214385
applicable:0.011320333
fiction:0.011318757
theological:0.011314333
write:0.011314122
polymath:0.011312485
not:0.01131248
Azerbaijan:0.011311639
Roads:0.011305569
Easter:0.011302605
Estonia:0.0112977885
 :0.011297341
Jr.:0.011295268
solution:0.011293906
anthem:0.01129315
systematic:0.011290461
bases:0.011287019
Virginia:0.011287
retired:0.011286539
lineup:0.011278584
Format:0.011271461
anthology:0.011270631
Carbon:0.011266927
gland:0.011262301
genius:0.01126016
lipids:0.011252144
boat:0.011231619
existed:0.011231435
aquarium:0.011230847
Eugene:0.011230672
weapon:0.011230532
synthesis:0.011223185
Olympic:0.011217556


Bott

# 2 - Compare Data Storage Methods

This section relates to content from **the week 1 lecture and the week 2 lab**.

Implement a variant of the model with a sparse vector for your input bag of words (See https://pytorch.org/docs/stable/sparse.html for how to switch a vector to be sparse). Use the default sparse vector type (COO).

In [8]:

linearRegression_new =  nn.Linear(7739,1)
optimizer = torch.optim.SGD(linearRegression_new.parameters(), lr=1e-4)


# Define the loss function
loss_func = F.mse_loss

# Calculate loss
loss = loss_func(linearRegression_new(x_data), y_data.unsqueeze(1))

x_data_sparse = x_data.to_sparse()
x_dev_data_sparse = bow_converter(dev_text, word_mapping(training_text)).to_sparse()
x_test_data_sparse = bow_converter(test_text, word_mapping(training_text)).to_sparse()

no_of_epochs = 100
display_interval = 10

for epoch in range(no_of_epochs):
  predictions1 = linearRegression_new(x_data_sparse)
  loss = loss_func(predictions1, y_data.unsqueeze(1))
  loss.backward()
  training_loss = mse(linearRegression_new(x_data_sparse), y_data.unsqueeze(1))   
  #print("Optimised:", "training loss=", "{:.9f}".format(training_loss.data))
  optimizer.step() #call step() to automatically update the parameters through our defined optimizer, which can be called once after backward()
  optimizer.zero_grad() #reset the gradient as what we did before
  if epoch % display_interval == 0 :
      
      # calculate the loss of the current model
      
      predictions2 = linearRegression_new(x_dev_data_sparse)
      loss = loss_func(predictions2, y_dev_data.unsqueeze(1))          
      print("Epoch:", '%04d' % (epoch), "dev loss=", "{:.8f}".format(loss))
  
testing_loss = loss_func(linearRegression_new(x_test_data), y_test_data.unsqueeze(1)) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))

Epoch: 0000 dev loss= 0.21340832
Epoch: 0010 dev loss= 0.16284272
Epoch: 0020 dev loss= 0.13214502
Epoch: 0030 dev loss= 0.11349096
Epoch: 0040 dev loss= 0.10213718
Epoch: 0050 dev loss= 0.09520841
Epoch: 0060 dev loss= 0.09096180
Epoch: 0070 dev loss= 0.08834108
Epoch: 0080 dev loss= 0.08670612
Epoch: 0090 dev loss= 0.08566905
Testing loss= 0.075715683
Absolute mean square loss difference: 0.007577106


## 2.1 Training and Test Speed [2pt]
Compare the time it takes to train and test the new model with the time it takes to train and test the old model.

You can time the execution of a line of code using `%time`.
See [this guide](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb#scrollTo=z1gyaC_PNZUB) for more on timing.

Checking the Training time and testing time for the Old Model First which uses x_data and y_data.


In [9]:
#Old Model Training Time
%%time

no_of_epochs = 100
display_interval = 10

for epoch in range(no_of_epochs):

  predictions = linearRegression(x_data)
  loss = loss_func(predictions, y_data.unsqueeze(1))
  loss.backward()
  training_loss = mse(linearRegression(x_data), y_data.unsqueeze(1))   
  #print("Optimised:", "training loss=", "{:.9f}".format(training_loss.data))
  optimizer.step() #call step() to automatically update the parameters through our defined optimizer, which can be called once after backward()
  optimizer.zero_grad() #reset the gradient as what we did before

CPU times: user 7.4 s, sys: 10.3 ms, total: 7.41 s
Wall time: 7.44 s


In [10]:
#Old Model Testing Time
%%time

testing_loss = loss_func(linearRegression(x_test_data), y_test_data.unsqueeze(1)) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))
                

Testing loss= 0.076088734
Absolute mean square loss difference: 0.009180911
CPU times: user 4.68 ms, sys: 21 µs, total: 4.7 ms
Wall time: 3.58 ms


New Model Training and Testing Time using Sparse Vectors

In [11]:
#New Model Training Time
%%time

no_of_epochs = 100
display_interval = 10

for epoch in range(no_of_epochs):
  predictions1 = linearRegression_new(x_data_sparse)
  loss = loss_func(predictions1, y_data.unsqueeze(1))
  loss.backward()
  training_loss = mse(linearRegression_new(x_data_sparse), y_data.unsqueeze(1))   
  #print("Optimised:", "training loss=", "{:.9f}".format(training_loss.data))
  optimizer.step() #call step() to automatically update the parameters through our defined optimizer, which can be called once after backward()
  optimizer.zero_grad() #reset the gradient as what we did before


CPU times: user 3.58 s, sys: 13.3 ms, total: 3.59 s
Wall time: 3.58 s


In [12]:
#New Model Testing Time
%%time


testing_loss = loss_func(linearRegression_new(x_test_data_sparse), y_test_data.unsqueeze(1)) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))

Testing loss= 0.074423753
Absolute mean square loss difference: 0.006830901
CPU times: user 3.11 ms, sys: 0 ns, total: 3.11 ms
Wall time: 2.65 ms


As we can see above, the training and testing time by using the sparse vector representation of the training data (new model) is lower than the training and testing time for the older model.

# 3 - Switch to Word Embeddings

This section relates to content from **the week 2 lecture and the week 3 lab**.

In this section, you will implement a model based on word2vec.

1. Use word2vec to learn embeddings for the words in your data.
2. Represent each input document as the average of the word vectors for the words it contains.
3. Train a linear regression model.

In [13]:
# First we create a tensor based on the word embeddings generated by Word2Vec.

import pprint
import re
import numpy as np

# For parsing our XML data
from lxml import etree 

# For data processing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

# For implementing the word2vec family of algorithms
from gensim.models import Word2Vec
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def exercise3 (t_text):
  normalized_text = []
  for string in t_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
  
  sentences=[]
  sentences=[word_tokenize(sentence) for sentence in normalized_text]
  model = Word2Vec(sentences, vector_size = 100 ,window=5, min_count=1, workers=4, sg=0)
  vocab, vectors = model.wv.key_to_index, model.wv.vectors

  doc_mean = []
  for doc in sentences:
    doc_mat = []
    for word in doc :
      if word in vocab:
        doc_mat.append(model.wv[word])
    if len(doc_mat) != 0:
      doc_mean.append(np.mean(doc_mat, axis=0))
    else:
      doc_mean.append(np.zeros(100))

   
  x = torch.Tensor(doc_mean)
  #x = x.mean(dim=1, keepdim=True)
  return x




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:

x = exercise3(training_text)
#import torch.optim as optim
# Define the linear regression model
l_model = nn.Linear(100, 1)

# Define the optimizer
optimizer = torch.optim.SGD(l_model.parameters(), lr=1e-4)

# Set the number of epochs
num_epochs = 100

# Train the model
for epoch in range(num_epochs):
    y_pred = l_model(x)
    loss = mse(y_pred, y_data)
    print("Optimised:", "training loss=", "{:.9f}".format(loss.data))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




<ipython-input-13-b1a0ac41abd3>:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  x = torch.Tensor(doc_mean)


Optimised: training loss= 0.451790839
Optimised: training loss= 0.448914379
Optimised: training loss= 0.446060508
Optimised: training loss= 0.443229139
Optimised: training loss= 0.440420061
Optimised: training loss= 0.437633127
Optimised: training loss= 0.434868097
Optimised: training loss= 0.432124883
Optimised: training loss= 0.429403275
Optimised: training loss= 0.426703125
Optimised: training loss= 0.424024224
Optimised: training loss= 0.421366394
Optimised: training loss= 0.418729603
Optimised: training loss= 0.416113526
Optimised: training loss= 0.413518012
Optimised: training loss= 0.410943031
Optimised: training loss= 0.408388227
Optimised: training loss= 0.405853599
Optimised: training loss= 0.403338939
Optimised: training loss= 0.400844038
Optimised: training loss= 0.398368895
Optimised: training loss= 0.395913213
Optimised: training loss= 0.393476814
Optimised: training loss= 0.391059607
Optimised: training loss= 0.388661504
Optimised: training loss= 0.386282235
Optimised: t

## 3.1 Accuracy [1pt]

Calculate the accuracy of your model.

In [15]:
# Your code goes here

x_test_new = exercise3(test_text)
y_test_new = torch.Tensor(test_lengths)
testing_loss = mse(l_model(x_test_new), y_test_new) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))

Testing loss= 0.181005016
Absolute mean square loss difference: 0.099750362


## 3.2 Speed [1pt]

Calcualte how long it takes your model to be evaluated.

In [16]:
# Your code goes here
%%time 

x_test = exercise3(test_text)
y_test = torch.Tensor(test_lengths)
testing_loss = mse(l_model(x_test), y_test) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))

Testing loss= 0.184044257
Absolute mean square loss difference: 0.102789603
CPU times: user 1.54 s, sys: 14.6 ms, total: 1.56 s
Wall time: 1.2 s


# 4 - Open-Ended Improvement

This section relates to content from **the week 1, 2, and 3 lectures and the week 1, 2, and 3 labs**.

This section is an open-ended opportunity to find ways to make your model more accurate and/or faster (e.g., use WordNet to generalise words, try different word features, other optimisers, etc).

We encourage you to try several ideas to provide scope for comparisons.

If none of your ideas work you can still get full marks for this section. You just need to justify the ideas and discuss why they may not have improved performance.


## 4.1 Ideas and Motivation [1pt]

In **this** box, describe your ideas and why you think they will improve accuracy and/or speed.

The main aim of my improvement is increasing the accuracy of the model. I have done that by building on section 3 of this assignment in which We average the word vectors from the vocabulary. The changes I have implemented are: 

1. Removing Stopwords, remove punctuation - In this during tokenisation, I removed some basic words which may affect the accuracy of the model since they occur quite frequently such as 'and' 'a'. Along with Stop words, I removed punctuations as well since we are interested in the actual words themselves. 

2. Fast Text : FastText was used since it accounts for out-of-vocabulary words (OOV). Since we were given the development set and test set for evaluation purposes, it contains new data that negatively affects the accuracy of the model and hence FastText allows us to deal with this better.


3. Increasing the vector size and epochs : In the previous sections, the epochs were limited to 100 and in this section the epochs has been set to 200. This  increases the training time but improves the accuracy.

4. Another minor change implemented which may have increased the accuracy is by changing the model algorithm from bow to skip gram.

For the improvement let us take the example of the word2vec representation and try to improve the model.

## 4.2 Implementation [2pt]

Implement your ideas

In [17]:
from typing import Text
# Your code goes here
from gensim.models import FastText
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

import re
import numpy as np

nltk.download('stopwords')
from nltk.corpus import stopwords as sw

def tokenize_doc(text):
  tokenized_doc = []
  for i in range(len(text)):
    # Tokenize sentences - for only doc1
    tokenized_sentence = sent_tokenize(text[i])
    #print("\ntokenized_sentence: ")
    #print(tokenized_sentence)

    # Remove punctuations - for only doc1
    punc_free_doc1 = re.sub(r'[^\w\s]','',text[i])
    #print("\npunc_free_sentence: ")
    #print(punc_free_doc1)

    # Tokenize words - for only doc1
    tokenized_doc1 = word_tokenize(punc_free_doc1)
    #print("\ntokenized_word: ")
    #print(tokenized_doc1)

    # Convert the tokens into lowercase: lower_tokens
    lower_tokens = [t.lower() for t in tokenized_doc1]
    #print("\nlower_case: ")
    #print(lower_tokens)

    # stop word removal
    sww = sw.words()
    tokenized_doc1 = [w for w in lower_tokens if not w in sww]
    #print("\ntokenized_word (in lower case, w/o stopwords): ")
    tokenized_doc.append(tokenized_doc1)
  return tokenized_doc



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
#Tokenizing the training set : 
def exercise4 (tokenized_doc):
  model = FastText(tokenized_doc, vector_size=200, window=5, min_count=5, workers=2, sg=1)

  
  vocab, vectors = model.wv.key_to_index, model.wv.vectors

  doc_mean = []
  for doc in tokenized_doc:
    doc_mat = []
    for word in doc :
      if word in vocab:
        doc_mat.append(model.wv[word])
    if len(doc_mat) != 0:
      doc_mean.append(np.mean(doc_mat, axis=0))
    else:
      doc_mean.append(np.zeros(100))

   
  x = torch.Tensor(doc_mean)
  #x = x.mean(dim=1, keepdim=True)
  return x


In [19]:
#Creating the tensor
training= tokenize_doc(training_text)
w = exercise4(training)

In [20]:

l_model = nn.Linear(200, 1)

# Define the optimizer
optimizer = torch.optim.SGD(l_model.parameters(), lr=1e-4)

# Set the number of epochs
num_epochs = 200

# Train the model
for epoch in range(num_epochs):
    y_pred = l_model(w)
    loss = mse(y_pred, y_data)
    #print("Optimised:", "training loss=", "{:.9f}".format(loss.data))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


## 4.3 Evaluation [1pt]

Evaluate the speed and accuracy of the model with your ideas

In [21]:
# Your code goes here
test = tokenize_doc(test_text)
x_test = exercise4(test)
y_test = torch.Tensor(test_lengths)
testing_loss = mse(l_model(x_test), y_test) 
print("Testing loss=", "{:.9f}".format(testing_loss.data))
print("Absolute mean square loss difference:", "{:.9f}".format(abs(
      training_loss.data - testing_loss.data)))

Testing loss= 0.137733027
Absolute mean square loss difference: 0.056478374


In **this** text box, briefly describe the results. Did your improvement work? Why / Why not?

The improvement worked slightly and better for newer vocabulary since it uses FastText.

I found that in this case increasing the vector size doesn't have much of an effect.

Otherwise the improvement is very minor and can be ignored. The accuracy is affected by the lack of similarity between the words in training, development and test.